In [1]:
import scipy as sp
import numpy as np
import sympy as smp

In [2]:
import math
def create_function(expression):
    t = smp.symbols('t')
    func = smp.lambdify(t, expression, 'numpy')
    return func



In [12]:

while True:
    try:
        rows =  int(input("ROWS: "))
        break
    except Exception:
        print("SHOULD BE INT ")
while True:
    try:        
        columns =int(input("COLUMNS: "))
        break
    except Exception:
        print("SHOULD BE INT ")  
inp = []
i = 0
while i < rows:
    
    data = input(f"Insert functions that depend on t or scalars as a row with length = {columns}: ").split()
    if len(data) != columns:
        print(f"ERROR: Row should have a length of {columns}.")
        continue

    try:
        tmp = lambda t, data=data: np.array([create_function(smp.sympify(expr))(t) for expr in data])
        inp.append(tmp)
        print(f"You've inserted a row {i + 1}")
        i += 1
    except smp.SympifyError:
        print("ERROR: Insert valid functions or scalars.")
        continue
            
A = lambda t: np.array([i(t) for i in inp])



ROWS: 3
COLUMNS: 2
Insert functions that depend on t or scalars as a row with length = 2: cos(t) sin(t)
You've inserted a row 1
Insert functions that depend on t or scalars as a row with length = 2: sin(t*34) cos(t)+2
You've inserted a row 2
Insert functions that depend on t or scalars as a row with length = 2: exp(t) t**3
You've inserted a row 3


In [13]:
while True:
    try:        
        data =input("B :  (len should = {})".format(rows)).split()
        if len(data) != rows:
            raise ValueError
        b = np.array([float(i) for i in data ])
        break
    except Exception:
        print("B :  (len should = {})".format(rows))  
b

B :  (len should = 3)2 2 2


array([2., 2., 2.])

In [14]:
while True:
    try:        
        data =input("v :  (len should = {})".format(columns)).split()
        if len(data) != columns:
            raise ValueError
        v = lambda t, data=data: np.array([create_function(smp.sympify(expr))(t) for expr in data]) 
        break
    except Exception:
        print("v :  (len should = {})".format(columns))  
v(1)        

v :  (len should = 2)sin(t) cos(t)


array([0.84147098, 0.54030231])

In [15]:
#A = lambda t:np.array([[t*50-1.23, np.sin(t)+2], [(t*2)**2 + np.pi, np.cos(t)], [np.exp(t), np.cos(t)**2]])
#b = np.array([1.4, -2.12, 3])
#v = lambda t: np.array([t*5-3, np.log(t)]) 

In [16]:
T =3

In [17]:
A(0)[0]

array([1., 0.])

In [18]:
from scipy.integrate import quad_vec

P = quad_vec(lambda tau: A(tau) @ A(tau).T, 0, T )[0]

In [19]:

x_hat= lambda tau: A(tau).T @ np.linalg.pinv(P) @ b
x_hat(3)#1.45

array([-0.75964271,  0.12943458])

In [20]:
Av = quad_vec(lambda tau: A(tau) @ v(tau).T, 0, T )[0]


In [21]:
Omega_xt =  lambda t : A(t).T @ np.linalg.pinv(P) @b + v(t) - A(t).T @ np.linalg.pinv(P) @ Av # 1.43

In [22]:
Omega_xt(3) 

array([-0.80922753, -0.69611285])

In [23]:
eps_sq = b.T @ b - b.T @ P @ np.linalg.pinv(P) @ b
eps_sq

3.552713678800501e-14

In [24]:
check1 = quad_vec(lambda tau: A(tau) @ Omega_xt(tau),0, 3)[0]
check1

array([2., 2., 2.])

In [25]:
check2 = quad_vec(lambda tau: A(tau) @ x_hat(tau),0, 3)[0]
check2

array([2., 2., 2.])

In [26]:
matrices_A_t = []
matrices_A_j = []
N = 4
length = np.array((A(1).T @ A(1))).shape[0]
def check(N):
    answ = np.zeros((length * (N ), length * (N )))
    # Compute matrices_A_t and matrices_A_j
    for i in range(N ):
        A_ti = A(T/(N-1) * i).T
        A_tj = A(T/(N-1) * i)
        matrices_A_t.append(np.array(A_ti))
        matrices_A_j.append(np.array(A_tj))
    for i in range(N ):
        for j in range(N):
            answ[i*length:(i+1)*length, j*length:(j+1)*length] = np.dot(matrices_A_t[i], matrices_A_j[j])
    determinant = np.linalg.det(answ)

    print("Determinant:", determinant)

    print(answ)
for i in range(2,N+1):
    check(i)

Determinant: 7.570628678627699e-10
[[  2.           0.          19.09554443  27.14112001]
 [  0.           9.           2.98448037   3.03002251]
 [ 19.09554443   2.98448037 405.39855898 543.1745717 ]
 [ 27.14112001   3.03002251 543.1745717  730.04003001]]
Determinant: 0.0
[[  2.           0.          19.09554443  27.14112001   2.
    0.        ]
 [  0.           9.           2.98448037   3.03002251   0.
    9.        ]
 [ 19.09554443   2.98448037 405.39855898 543.1745717   19.09554443
    2.98448037]
 [ 27.14112001   3.03002251 543.1745717  730.04003001  27.14112001
    3.03002251]
 [  2.           0.          19.09554443  27.14112001   2.
    0.        ]
 [  0.           9.           2.98448037   3.03002251   0.
    9.        ]]
Determinant: 0.0
[[  2.           0.          19.09554443  27.14112001   2.
    0.           4.55242627   4.37249499]
 [  0.           9.           2.98448037   3.03002251   0.
    9.           2.01068753   6.21221161]
 [ 19.09554443   2.98448037 405.39855898 